In [1]:
import numpy as np
import random

In [2]:
def look_then_leap(look_phase_share: float, candidate_scores: np.typing.NDArray[np.int_], candidate_rejection_probs: np.typing.NDArray[np.double]) -> int:
    assert 0 < look_phase_share < 1
    exploration_end =  int(len(candidate_scores)*look_phase_share)
    explored_candidate_scores = candidate_scores[:exploration_end]
    max_explored_candidate_score = explored_candidate_scores.max()
    candidate_scores[:exploration_end] = 0  # we reject explored candidates
    no_candidate_accepted = True
    leap_id = len(candidate_scores) - 1  # in worst case leap for the last candidate
    while no_candidate_accepted:
        candidates_better_than_max_explored = candidate_scores > max_explored_candidate_score
        # leap for the first candidate that is better than what was seen during exploration
        if np.any(candidates_better_than_max_explored):
            leap_id = np.where(candidates_better_than_max_explored)[0][0]
        no_candidate_accepted = candidate_rejection_probs[leap_id] >= random.random()
        if no_candidate_accepted:
            candidate_scores[leap_id] = 0  # candidate rejected
            # only candidate left is the last one and the last one rejected
            if np.all(candidates_better_than_max_explored) == False:
                return 0
    return int(candidate_scores[leap_id])

In [3]:
import pandas as pd

n_runs = 1000

# without rejection
static_rejection_prob = 0
look_phase_share = 0.37
eval_df_dict = {}
for n_candidates in [5, 10, 20, 100, 1000, 10_000, 100_000]:
    candidate_rejection_probs = np.ones(n_candidates) * static_rejection_prob
    achieved_scores = np.zeros(n_runs, dtype=int)
    for run in range(n_runs):
        candidate_scores = np.random.permutation(np.arange(start=1, step=1, stop=n_candidates+1))
        achieved_scores[run] = look_then_leap(look_phase_share=look_phase_share, candidate_scores=candidate_scores, candidate_rejection_probs=candidate_rejection_probs)
    eval_df_dict[f"{n_candidates} candidates"] = achieved_scores / n_candidates

eval_df = pd.DataFrame(eval_df_dict)
mean_relative_score = eval_df.mean()
share_best = (eval_df == 1).sum() / n_runs
share_top_10_percent = (eval_df >= 0.9).sum() / n_runs
df_results = pd.concat([mean_relative_score, share_best, share_top_10_percent], axis=1)
df_results.columns = ["average score", "share of runs resulting in best score", "share of runs resulting in top 10 % score"]
df_results

,average score,share of runs resulting in best score,share of runs resulting in top 10 % score
5 candidates,0.790000,0.438,0.438
10 candidates,0.795200,0.394,0.598
20 candidates,0.798350,0.401,0.627
100 candidates,0.821290,0.353,0.689
1000 candidates,0.796221,0.335,0.642
10000 candidates,0.812890,0.367,0.662
100000 candidates,0.813153,0.356,0.662


In [4]:
# with rejection + same exploration phase length
static_rejection_prob = 0.3
look_phase_share = 0.37  
eval_df_dict = {}
for n_candidates in [5, 10, 20, 100, 1000, 10_000, 100_000]:
    candidate_rejection_probs = np.ones(n_candidates) * static_rejection_prob
    achieved_scores = np.zeros(n_runs, dtype=int)
    for run in range(n_runs):
        candidate_scores = np.random.permutation(np.arange(start=1, step=1, stop=n_candidates+1))
        achieved_scores[run] = look_then_leap(look_phase_share=look_phase_share, candidate_scores=candidate_scores, candidate_rejection_probs=candidate_rejection_probs)
    eval_df_dict[f"{n_candidates} candidates"] = achieved_scores / n_candidates

eval_df = pd.DataFrame(eval_df_dict)
mean_relative_score = eval_df.mean()
share_best = (eval_df == 1).sum() / n_runs
share_top_10_percent = (eval_df >= 0.9).sum() / n_runs
df_results = pd.concat([mean_relative_score, share_best, share_top_10_percent], axis=1)
df_results.columns = ["average score", "share of runs resulting in best score", "share of runs resulting in top 10 % score"]
df_results

,average score,share of runs resulting in best score,share of runs resulting in top 10 % score
5 candidates,0.537000,0.296,0.296
10 candidates,0.549600,0.274,0.407
20 candidates,0.545450,0.250,0.426
100 candidates,0.565090,0.251,0.459
1000 candidates,0.577184,0.272,0.485
10000 candidates,0.569364,0.257,0.469
100000 candidates,0.548910,0.250,0.446
